<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 코스피/코스닥 시총 100순위 변동
- 1주일/1달 주도주 분석
- 상승률 변동 (20, 60, 120, 250) 
- 강한 테마/이슈 뉴스검색어와 분석으로 연속성 검증
- 조회수순? 영향력 강한 1주일 이내 공시 분석기
- 앞으로 상장할 주식 1주일 치 이슈에 부합하나
- 업황 분석기: 지난 1주 1달치 업황 상승률/ 거래대금 순위

In [15]:
import requests
from datetime import date
import pandas as pd
from io import StringIO

In [16]:
market_type = {
    'all':'ALL',
    'kospi':'STK',
    'kosdaq':'KSQ'
}

def get_ranking(market, ranking_type, start_date, end_date=date.today().strftime("%Y%m%d")):
  gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
  url = ''
  if ranking_type == 'highest': url = 'dbms/MDC/EASY/ranking/MDCEASY01501'
  elif ranking_type == 'traded': url = 'dbms/MDC/EASY/ranking/MDCEASY01601'
  elif ranking_type == 'marketcap': url = 'dbms/MDC/EASY/ranking/MDCEASY01701'
  data = {
    'locale': 'ko_KR',
    'mktId': market,
    'itmTpCd': '1',
    'itmTpCd2': '1',
    'trdDd': start_date,
    'strtDd': start_date,
    'endDd': end_date,
    'stkprcTpCd': 'Y',
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': url
  }
  gen_key = requests.post(gen_url, data=data).text
  down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  data = {'code':gen_key}
  r = requests.post(down_url, data=data)
  r.encoding = 'EUC-KR'
  return pd.read_csv(StringIO(r.text))

코스피/코스닥 시총 100순위 변동

In [17]:
df = pd.DataFrame()
for d in pd.date_range(start='2021/01/01', periods=19, freq='1MS'):
  t = pd.DataFrame()
  dt = d
  while t.empty:
    t = get_ranking(market_type['kosdaq'], 'marketcap', dt.strftime('%Y%m%d'))[['순위', '종목명']]
    t['date'] = dt
    dt += pd.DateOffset(1)
  df = df.append(t, ignore_index=True)

In [18]:
df

,순위,종목명,date
0,1,셀트리온헬스케어,2021-01-04
1,2,셀트리온제약,2021-01-04
2,3,에이치엘비,2021-01-04
3,4,알테오젠,2021-01-04
4,5,씨젠,2021-01-04
...,...,...,...
945,46,고영,2022-07-01
946,47,아프리카TV,2022-07-01
947,48,메가스터디교육,2022-07-01
948,49,NICE평가정보,2022-07-01


In [19]:
i = df[df['순위'].isin(range(10,51))].index
t_df = df.drop(i)

In [20]:
!pip install colour

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import altair as alt
import pandas as pd
from colour import Color

source = t_df
len = t_df['종목명'].nunique()

red = Color("red")
colors = list(red.range_to(Color("green"),len))

alt.Chart(source).mark_line(point = True).encode(
    x = alt.X("date:O", timeUnit="yearmonthdate", title="date"),
    y="rank:O",
    color=alt.Color("종목명")
).transform_window(
    rank="rank()",
    sort=[alt.SortField("순위", order="ascending")],
    groupby=["date"]
).properties(
    width=1000,
    height=500,
)

alt.Chart(...)

In [22]:
import altair as alt
import pandas as pd
from colour import Color
import random

source = t_df
len = t_df['종목명'].nunique()
print()

colors = [x.hex for x in list(Color("red").range_to(Color("green"),len))]

alt.Chart(source).mark_line(point = True).encode(
    x = alt.X("date:O", timeUnit="yearmonthdate", title="date"),
    y = alt.Y("rank:N"),
    color = alt.Color("종목명:N", scale={"range": colors}),
    strokeWidth = alt.condition("datum", alt.value(2.5), alt.value(1))
).transform_window(
    rank="rank()",
    sort=[alt.SortField("순위", order="ascending")],
    groupby=["date"]
).properties(
    width=1000,
    height=500,
)

alt.Chart(...)

In [23]:
source

,순위,종목명,date
0,1,셀트리온헬스케어,2021-01-04
1,2,셀트리온제약,2021-01-04
2,3,에이치엘비,2021-01-04
3,4,알테오젠,2021-01-04
4,5,씨젠,2021-01-04
...,...,...,...
904,5,펄어비스,2022-07-01
905,6,셀트리온제약,2022-07-01
906,7,에코프로비엠,2022-07-01
907,8,알테오젠,2022-07-01


In [24]:
# korean font
!wget 'https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKkr-hinted.zip'
!unzip "NotoSansCJKkr-hinted.zip"
!mv NotoSansCJKkr-Medium.otf /usr/share/fonts/truetype/

--2022-07-12 13:18:37--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKkr-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121163981 (116M) [application/zip]
Saving to: ‘NotoSansCJKkr-hinted.zip.3’

NotoSansCJKkr-hinte 100%[===================>] 115.55M   122MB/s    in 0.9s    

2022-07-12 13:18:38 (122 MB/s) - ‘NotoSansCJKkr-hinted.zip.3’ saved [121163981/121163981]

Archive:  NotoSansCJKkr-hinted.zip
replace LICENSE_OFL.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: LICENSE_OFL.txt         
  inflating: NotoSansCJKkr-Black.otf  
  inflating: NotoSansCJKkr-Bold.otf  
  inflating: NotoSansCJKkr-DemiLight.otf  
  inflating: NotoSansCJKkr-Light.otf  
  inflating: NotoSansCJKkr-Medium.otf  
  inf

In [34]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

font_path = 'Inter-Regular.otf'  # Your font path goes here
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()

In [35]:
import matplotlib.font_manager as fm

font_list = fm.fontManager.addfont('NotoSansMonoCJKkr-Regular.otf')
fm.fontManager.ttflist.extend(font_list)
plt.rcParams['font.family'] = 'Noto Sans CJK KR'

FileNotFoundError: ignored

In [25]:
n_top_ranked = 10
top_sources = df[df["date"] == df["date"].max()].nsmallest(n_top_ranked, "순위")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FixedFormatter, FixedLocator

fig, ax = plt.subplots(figsize=(120, 80), subplot_kw=dict(ylim=(0.5, 0.5 + n_top_ranked)))

ax.xaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(1))

yax2 = ax.secondary_yaxis("right")
yax2.yaxis.set_major_locator(FixedLocator(top_sources["순위"].to_list()))
yax2.yaxis.set_major_formatter(FixedFormatter(top_sources["종목명"].to_list()))

for i, j in df.groupby("종목명"):
    ax.plot("date", "순위", "o-", data=j, mfc="w")

ax.invert_yaxis()
ax.set(xlabel="Edition", ylabel="Rank", title="Popularity of publisher by edition")
ax.grid(axis="x")
plt.tight_layout()